<a href="https://colab.research.google.com/github/revendrat/Big-Data-Analytics/blob/main/05_Data_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agenda
* Computing Mean/Min/Max values of an array
* Counting Occurrences of Elements
* Applying arithmetic functions to arrays.
* Appending tables to an existing table
* Adding a column to an existing Table
* Searching for values matching a predicate in Arrays

## Computing Mean/Min/Max values of an array

In [1]:
# import package-modules
import pyarrow as pa
import pyarrow.compute as pc

In [2]:
#declare array
a = pa.array([1, 1, 2, 3, 7, 5, 6, 6, 3])

In [3]:
# calculate sum of array 
pc.sum(a)

<pyarrow.Int64Scalar: 34>

In [4]:
# calculate mean of array
pc.mean(a)

<pyarrow.DoubleScalar: 3.7777777777777777>

In [5]:
# calculate min-max values of array
pc.min_max(a)

<pyarrow.StructScalar: [('min', 1), ('max', 7)]>

## Counting Occurances of Elements

In [6]:
pc.value_counts(a)

-- is_valid: all not null
-- child 0 type: int64
  [
    1,
    2,
    3,
    7,
    5,
    6
  ]
-- child 1 type: int64
  [
    2,
    1,
    2,
    1,
    1,
    2
  ]

In [7]:
count_a = pc.value_counts(a)
for i in count_a:
  print(i)

[('values', 1), ('counts', 2)]
[('values', 2), ('counts', 1)]
[('values', 3), ('counts', 2)]
[('values', 7), ('counts', 1)]
[('values', 5), ('counts', 1)]
[('values', 6), ('counts', 2)]


## Apply arithmetic functions to arrays

In [8]:
# multiply all values in array by 2
pc.multiply(a,2)

[
  2,
  2,
  4,
  6,
  14,
  10,
  12,
  12,
  6
]

In [9]:
# divide all values in array by 2
pc.divide(a,2)

[
  0,
  0,
  1,
  1,
  3,
  2,
  3,
  3,
  1
]

## Append tables to an existing table
* Say two arrow tables containing analytics courses and business courses are created.
* Append or combine the tables using pyarrow.concat_tables()
* Appending tables is a zero-copy operation in Arrow
* The data is copied during append operations only when casting of data types is performed

In [10]:
analytics_courses = pa.table([["Programming for Analytics", "Big Data Analytics"], ["MPBA 517", "MPBA 519"]], names = ["Course Name", "Course Code"])

In [11]:
analytics_courses

pyarrow.Table
Course Name: string
Course Code: string
----
Course Name: [["Programming for Analytics","Big Data Analytics"]]
Course Code: [["MPBA 517","MPBA 519"]]

In [12]:
business_courses = pa.table([["Strategic Management", "Entrepreneurship"], ["MPA 410", "MBA 411"]], names = ["Course Name", "Course Code"])

In [13]:
business_courses

pyarrow.Table
Course Name: string
Course Code: string
----
Course Name: [["Strategic Management","Entrepreneurship"]]
Course Code: [["MPA 410","MBA 411"]]

In [14]:
combined_courses = pa.concat_tables([analytics_courses, business_courses])
combined_courses

pyarrow.Table
Course Name: string
Course Code: string
----
Course Name: [["Programming for Analytics","Big Data Analytics"],["Strategic Management","Entrepreneurship"]]
Course Code: [["MPBA 517","MPBA 519"],["MPA 410","MBA 411"]]

## Adding a column to an existing Table
* suppose you need to add semester coulmn to tables: combined_courses
* use pyarrow.Table.append_column()

In [16]:
print(combined_courses)

pyarrow.Table
Course Name: string
Course Code: string
----
Course Name: [["Programming for Analytics","Big Data Analytics"],["Strategic Management","Entrepreneurship"]]
Course Code: [["MPBA 517","MPBA 519"],["MPA 410","MBA 411"]]


In [20]:
# Append operation
combined_courses = combined_courses.append_column("Semester", pa.array(["I", "II", "II", "III"]))
combined_courses

pyarrow.Table
Course Name: string
Course Code: string
Semester: string
----
Course Name: [["Programming for Analytics","Big Data Analytics"],["Strategic Management","Entrepreneurship"]]
Course Code: [["MPBA 517","MPBA 519"],["MPA 410","MBA 411"]]
Semester: [["I","II","II","III"]]

## Replacing a column in an existing Table
* Assume that you have a table with vegetables and their prices in dollars per kilogram.
* You want to replace prices column with prices in rupees
* use pyarrow.Table.set_column() method
* make sure to use appropriate index number (1 for 2nd column) to replace the column

In [21]:
veg_prices = pa.table([["Tomatos", "Potatoes", "Beans", "Red Chillies"],
                      [2,4,6,3]], names = ["veg_item", "price_usd"])
veg_prices

pyarrow.Table
veg_item: string
price_usd: int64
----
veg_item: [["Tomatos","Potatoes","Beans","Red Chillies"]]
price_usd: [[2,4,6,3]]

In [22]:
# replace column
new_veg_prices = veg_prices.set_column(
    1,
    "price_inr",
    pa.array([2*75, 4*75, 6*75, 3*75])
)
new_veg_prices

pyarrow.Table
veg_item: string
price_inr: int64
----
veg_item: [["Tomatos","Potatoes","Beans","Red Chillies"]]
price_inr: [[150,300,450,225]]

## Searching for values matching a predicate in Arrays
* In case you are looking for a matching value in Arrow arrays, the arrow.compute module has several methods
* Illustrations using greater, lower, and equal to a number are provided below

In [23]:
num_arr = pa.array(range(10))
num_arr

[
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9
]

In [25]:
# Fetch all values in num_arr that are greater than 5
# pc.greater(num_arr,5)
five_label = pc.greater(num_arr,5)
greater_five = pc.filter(num_arr, five_label)
greater_five

[
  6,
  7,
  8,
  9
]

In [28]:
# Fetch all values in num_arr that are lower than 5
#pc.less(num_arr,5)
five_label = pc.less(num_arr,5)
lower_five = pc.filter(num_arr, five_label)
lower_five

[
  0,
  1,
  2,
  3,
  4
]

In [29]:
# Fetch all values in num_arr that are equal 5
#pc.equal(num_arr,5)
five_label = pc.equal(num_arr,5)
equal_five = pc.filter(num_arr, five_label)
equal_five

[
  5
]